In [1]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="OpenBuddy/openbuddy-llama2-70b-v10.1-bf16")

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("OpenBuddy/openbuddy-llama2-70b-v10.1-bf16")
model = AutoModelForCausalLM.from_pretrained("OpenBuddy/openbuddy-llama2-70b-v10.1-bf16")

In [ ]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

In [ ]:
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

In [ ]:
# Hub Model configuration. https://huggingface.co/models
hub = {
	'HF_MODEL_ID':'OpenBuddy/openbuddy-llama2-70b-v10.1-bf16',
	'SM_NUM_GPUS': json.dumps(1)
}

In [ ]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="1.0.3"),
	env=hub,
	role=role, 
)

# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.2xlarge",
	container_startup_health_check_timeout=300,
  )
  
# send request
predictor.predict({
	"inputs": "Hallo kannst du mit mir auf Deutsch kommunizieren?",
})

In [ ]:
# delete endpoint
predictor.delete_model()
predictor.delete_endpoint()